In [1]:
# Constants
from config.execution_parameters import *

# Project libraries
from src.simulator_package.simulator_functions import *
import src.hmm_package.generate_hmm
from src.hmm_package.generate_hmm import *
from src.plot_and_print_info.plots_and_print_info import *

import random
import importlib

In [11]:
importlib.reload(src.hmm_package.generate_hmm)
from src.hmm_package.generate_hmm import *

In [5]:
advertising_campaign = SimulationClass(ANALYSIS_MODE)
advertising_campaign.simulate()

adstock = compute_adstock(observation=advertising_campaign.results["user_expositions"])
output = np.append( np.zeros([adstock.shape[0], 1]), advertising_campaign.results["user_outcome"], axis=1 )

In [6]:
adstock.shape

TensorShape([12626, 3, 31])

In [5]:
empty_users = 6000
adstock_with_empty = tf.concat([adstock, tf.zeros([empty_users, adstock.shape[1], adstock.shape[2]], dtype=float)], axis = 0)
output_with_empty = np.append(output, np.zeros([empty_users, adstock.shape[2]]), axis = 0)

(12188, 31)

In [8]:
empty_users = 5000
adstock_only_empty = tf.zeros([empty_users, adstock.shape[1], adstock.shape[2]], dtype=float)
output_only_empty = np.zeros([empty_users, adstock.shape[2]])

# Lets add a 3% of conversion due to external factors
for temp in range(round(0.03*empty_users)):
    output_only_empty[temp, random.randint(1,adstock.shape[2]):] = 1


In [12]:
MU = [ -2.7419195, -7.4647417, -1.1010361, -5.92728  ]
model = build_hmm_to_fit_beta( states_observable=STATES_ARE_OBSERVABLE, mu=MU )

compiler = CompilerInfoBeta(LR_EXPONENTIAL_DECAY)
model.compile(
    loss = compiler.loss,
    optimizer = compiler.optimizer,
    run_eagerly = True
)

#fit_model(model, adstock, output)
fit_model(model, adstock, output)

Beta: [array([-0.2528566 ,  0.1959858 ,  0.23891842, -0.30465543,  0.42454708,
       -0.27133608,  0.25765383,  0.43076503, -0.20775962,  0.4620564 ,
       -0.03529656,  0.0498271 ], dtype=float32)]
Epoch 1/10000
51/51 [==============================] - 84s 2s/step - loss: 377.0730
Beta: [array([-0.20107226,  0.24683352,  0.18611048, -0.25145397,  0.4761612 ,
       -0.22030638,  0.2055543 ,  0.48345813, -0.16921237,  0.51218545,
       -0.08697189,  0.1017909 ], dtype=float32)]
Epoch 2/10000
51/51 [==============================] - 82s 2s/step - loss: 365.7585
Beta: [array([-0.14135803,  0.2940745 ,  0.13057911, -0.19395177,  0.53116673,
       -0.1745625 ,  0.15232125,  0.53862613, -0.11434188,  0.5585369 ,
       -0.14190851,  0.15820615], dtype=float32)]
Epoch 3/10000
51/51 [==============================] - 88s 2s/step - loss: 354.8901
Beta: [array([-0.0696855 ,  0.3376508 ,  0.06937398, -0.12876624,  0.5929885 ,
       -0.13398919,  0.09556693,  0.5994662 , -0.046658  ,  0.6002

KeyboardInterrupt: 

In [10]:
fit_model(model, adstock_with_empty, output_with_empty)

NameError: name 'adstock_with_empty' is not defined

In [31]:
t = make_transition_matrix(model.weights[0], model.weights[1], adstock_with_empty[-2:-1,:,:])
hmm = tfd.HiddenMarkovModel(
            initial_distribution=generate_hmm_distributions(transition_matrix=t, states_observable=False)['initial_distribution'],
            transition_distribution=tfd.Categorical(probs=t),
            observation_distribution=generate_hmm_distributions(transition_matrix=t, states_observable=False)['observation_distribution'],
            time_varying_transition_distribution=True,
            num_steps=execution_duration + 1)

In [77]:
make_non_exposed_user_transtion_matrix(model.weights[0],1000)

<tf.Tensor: shape=(30, 3, 3), dtype=float32, numpy=
array([[[9.9954051e-01, 3.6151949e-04, 9.7974400e-05],
        [4.9749774e-01, 4.9749786e-01, 5.0044274e-03],
        [1.0000000e-06, 1.0000000e-06, 9.9999797e-01]],

       [[9.9954051e-01, 3.6151949e-04, 9.7974400e-05],
        [4.9749774e-01, 4.9749786e-01, 5.0044274e-03],
        [1.0000000e-06, 1.0000000e-06, 9.9999797e-01]],

       [[9.9954051e-01, 3.6151949e-04, 9.7974400e-05],
        [4.9749774e-01, 4.9749786e-01, 5.0044274e-03],
        [1.0000000e-06, 1.0000000e-06, 9.9999797e-01]],

       [[9.9954051e-01, 3.6151949e-04, 9.7974400e-05],
        [4.9749774e-01, 4.9749786e-01, 5.0044274e-03],
        [1.0000000e-06, 1.0000000e-06, 9.9999797e-01]],

       [[9.9954051e-01, 3.6151949e-04, 9.7974400e-05],
        [4.9749774e-01, 4.9749786e-01, 5.0044274e-03],
        [1.0000000e-06, 1.0000000e-06, 9.9999797e-01]],

       [[9.9954051e-01, 3.6151949e-04, 9.7974400e-05],
        [4.9749774e-01, 4.9749786e-01, 5.0044274e-03],
   

In [14]:
make_transition_matrix(MU, model.weights, adstock[10:11,:,:])

<tf.Tensor: shape=(1, 30, 3, 3), dtype=float32, numpy=
array([[[[7.58311272e-01, 2.41560563e-01, 1.28200336e-04],
         [2.64219254e-01, 7.23513424e-01, 1.22673288e-02],
         [1.00000001e-10, 1.00000001e-10, 1.00000000e+00]],

        [[1.42074943e-01, 8.57896984e-01, 2.80693694e-05],
         [1.91862099e-02, 9.36125278e-01, 4.46885377e-02],
         [1.00000001e-10, 1.00000001e-10, 1.00000000e+00]],

        [[2.91064441e-01, 7.08864391e-01, 7.11511311e-05],
         [3.36762033e-02, 9.41095591e-01, 2.52282154e-02],
         [1.00000001e-10, 1.00000001e-10, 1.00000000e+00]],

        [[4.59102154e-01, 5.40764749e-01, 1.33071298e-04],
         [5.20960540e-02, 9.32154953e-01, 1.57490131e-02],
         [1.00000001e-10, 1.00000001e-10, 1.00000000e+00]],

        [[6.02743745e-01, 3.97056073e-01, 2.00213530e-04],
         [7.31455162e-02, 9.16155457e-01, 1.06990281e-02],
         [1.00000001e-10, 1.00000001e-10, 1.00000000e+00]],

        [[7.07116604e-01, 2.92621493e-01, 2.619390

In [15]:
model.weights

[<tf.Variable 'transition_prob_layer_beta_2/beta:0' shape=(12,) dtype=float32, numpy=
 array([ 1.630846  , -0.04413975, -1.4307394 ,  0.7025837 , -0.03290017,
        -1.1763734 ,  1.5244356 ,  1.147517  , -0.30141446, -1.0168557 ,
         1.7960523 ,  1.1943513 ], dtype=float32)>]

In [80]:
a = make_transition_matrix(model.weights[0], model.weights[1], adstock_with_empty[-2:-1,:,:])
np.linalg.matrix_power( a[0,0,:].numpy(), 30 )

array([[9.9624962e-01, 7.1688538e-04, 3.0336005e-03],
       [9.8652732e-01, 7.0989016e-04, 1.2762990e-02],
       [5.7610592e-09, 2.0286492e-10, 1.0000000e+00]], dtype=float32)

In [ ]:
output_with_empty

In [82]:
np.linalg.matrix_power( a[0,0,:].numpy(), 30 )[0,2]

0.0030336005